# Lab 5 - Projectile Motion I
        May 24, 2018 & May 31 2018
        
[<< Table of Contents](index.html)

## P5.1
In this exercise you will write a Python function that calculates the range of
a particle. In other words, this function will answer the question: "After my
projectile has traveled $h$ meters vertically, where will it be located horizontally?"

##### (a) Derive the equation for the range of the projectile. First work in the y-direction to determine the time of flight. Next, use this time of flight to find the horizontal range of the projectile. Set the launch position as your origin and let $y_f$ be the height of the target. You should find that:

<sub>5.4</sub>
$$
x_f = v \cos{\theta}\Bigg[ \frac{v\sin{\theta} \sqrt{v^2 \sin^2{\theta} - 2gy_f}}{g} \Bigg]
$$

![Scratch Work](lab5_fig1.jpg)


##### (b) Read section 4.3 in the python book to remind yourself about python functions.
> I know Python better than I know english, so I did not feel the need to read.

##### (c) Write a python function that evaluates equation (5.4) and returns the range of the particle and the flight time. The function should take the launch speed, launch angle, and target height as arguments. The function should return the flight time and the horizontal landing location of the projectile.

In [31]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# speed {m/s}, angle {degrees}, target_height {meters}
def _time_of_flight(speed, angle, target_height):
    v = speed * np.sin(np.radians(angle))
    g = 9.81 # m/s^2
    return (v + np.sqrt(v**2 - 2*g*target_height))/g

# speed {m/s}, angle {degrees}, target_height {meters}
def _range(speed, angle, time):
    return speed * np.cos(np.radians(angle)) * time

# speed {m/s}, angle {degrees}, time {seconds}
def eval_flight(speed, angle, target_height):
    t = _time_of_flight(speed, angle, target_height)
    r = _range(speed, angle, t)
    return { "time" : t, "range": r }


##### (d) Call your function using the values given to check that it is working correctly.

In [32]:
velocities = [4.5, 8.2, 8.5, 8.5, 4.5]
angles = [60, 60, 30, 60, 30]
target_heights = [0.75, 0.75, 0.75, 0.75, -0.75]
ranges = []

for s, a, h in zip(velocities, angles, target_heights):
    print("TESTING: ", s, a, h)
    ranges.append(eval_flight(s, a, h))
    print(ranges[-1])


TESTING:  4.5 60 0.75
{'time': 0.46732932428882246, 'range': 1.0514909796498508}
TESTING:  8.2 60 0.75
{'time': 1.3330898184338456, 'range': 5.465668255578768}
TESTING:  8.5 30 0.75
{'time': 0.6197367427475226, 'range': 4.562015984462796}
TESTING:  8.5 60 0.75
{'time': 1.3908185844757357, 'range': 5.910978984021878}
TESTING:  4.5 30 -0.75
{'time': 0.682690124782067, 'range': 2.660521459383173}


> the above values match the results in the book

## P5.2


##### (a) Pick one of these derivatives (found in the book) and verify that it is correct. Record the pencil-and-paper work in your lab notebook.

$$
x_f = v \cos{\theta}\Bigg[ \frac{v\sin{\theta} \sqrt{v^2 \sin^2{\theta} - 2gy_f}}{g} \Bigg]
$$
<br>

$$
\frac{\delta x_f}{\delta y_f}= \frac{\delta}{\delta y_f} v \cos{\theta}\Bigg[ \frac{v\sin{\theta} \sqrt{v^2 \sin^2{\theta} - 2gy_f}}{g} \Bigg]
$$
<br>

$$
\frac{\delta x_f}{\delta y_f}= \frac{\delta}{\delta y_f} 
    \frac{v^2 \cos{\theta} \sin{\theta}}{g} + \frac{v\cos{\theta} \big(v^2 \sin^2{\theta} - 2gy_f)^{\frac{1}{2}}}{g}  
$$
<br>

$$
\frac{\delta x_f}{\delta y_f}= \frac{\delta}{\delta y_f} 
    \frac{2 g v\cos{\theta}}{2g \big(v^2 \sin^2{\theta} - 2gy_f)^{\frac{1}{2}}}  
$$
<br>


$$
\frac{\delta x_f}{\delta y_f}= \frac{\delta}{\delta y_f} 
    \frac{v\cos{\theta}}{\big(v^2 \sin^2{\theta} - 2gy_f)^{\frac{1}{2}}}  
$$
<br>
<br>

> this matches the result (5.9) in the book

##### (b) Use equations (5.5) - (5.9) to construct a Python function to calculate the uncertainty in the range.

In [33]:
class PartialDerivative:
    # v (velocity), t (angle), y (height)
    def __init__(self, v, t, y):
        self.g = 9.8 # m/s^2
        self.v = v
        self.v2 = v**2
        self.t = t
        self.y = y
        self.theta = np.radians(t)
        self.sin = np.sin(self.theta)
        self.cos = np.cos(self.theta)
        self.v_sin = v * self.sin
        self.v_cos = v * self.cos
        self.sin2 = self.sin**2
        self.square = np.sqrt((self.v2 * self.sin2) - (2*self.g*y))
        

    def with_respect_to_velocity(self):
        term1 = self.cos*((self.v_sin + self.square)/self.g)
        term2 = (self.v_cos/self.g) *(self.sin + (self.v*self.sin2/self.square))
        return term1 + term2
    
    
    def with_respect_to_theta(self):
        term1 = self.v_sin * ((self.v_sin + self.square) / self.g)
        term2 = (self.v_cos/self.g)*(self.v_cos + ((self.v2*self.sin*self.cos)/self.square))
        return -term1 + term2
    
    
    def with_respect_to_height(self):
        return - (self.v_cos / self.square)

def rangeUncertainty(v,deltaV, theta,deltaTheta, finalHeight,deltaHeight):
    PD = PartialDerivative(v, theta, finalHeight)
    a = (PD.with_respect_to_velocity() * deltaV)**2
    b = (PD.with_respect_to_theta() * np.radians(deltaTheta))**2
    c = (PD.with_respect_to_height() * deltaHeight)**2
    return np.sqrt(a + b + c)

##### (c) Call your function using the values below to verify that it is working correctly.

In [34]:
# sets of tuples {(value, uncertainty)}
vel = [(4.5,.03), (8.2,.03), (8.5,.03), (8.5,.03), (4.5,.03)]
ang = [(60,1.0), (60,1.0), (30,0.5), (30,0.1), (30,1.0)]
y_f = [(0.75,0.025), (0.75,0.025), (0.75,0.01), (0.75,0.01), (-0.75,0.025)]

for v, a, y in zip(vel, ang, y_f):
    un = rangeUncertainty(v[0],v[1], a[0],a[1], y[0],y[1])
    print ("RANGE UNCERTAINTY: {}±{}, {}±{}, {}±{} ==> {}".format(v[0],v[1], a[0],a[1], y[0],y[1], un))

RANGE UNCERTAINTY: 4.5±0.03, 60±1.0, 0.75±0.025 ==> 0.09688466895508206
RANGE UNCERTAINTY: 8.2±0.03, 60±1.0, 0.75±0.025 ==> 0.11042641610909386
RANGE UNCERTAINTY: 8.5±0.03, 30±0.5, 0.75±0.01 ==> 0.15251650901236571
RANGE UNCERTAINTY: 8.5±0.03, 30±0.1, 0.75±0.01 ==> 0.07229290537493593
RANGE UNCERTAINTY: 4.5±0.03, 30±1.0, -0.75±0.025 ==> 0.037262357842082657


> These values match the numbers in the book

## P5.3
Now we need to search for the angle that will cause the projectile to hit the
target. The idea here is to consider a large number of launch angles. For
each launch angle we check to see if the target was hit. If it was, we stop, if
we didn’t let’s keep checking more launch angles. This idea is depicted in
figures 5.1 and 5.2. Follow the steps below to implement this.

##### (a) Read chapter 6 in the python book to learn about loops and logic.
> I don't need to read the chapter, I am VERY familiar with loops and logic.

#####  (b) To search for the right launch angle, let’s pick a range of launch angles and loop over them. For each launch angle, we’ll calculate the landing location of the projectile and it’s associated uncertainty. If the target’s location falls within the uncertainty window, we’ll stop. If it didn’t, we’ll keep going with the loop until it does.

In [35]:
def hit_the_target(velocity, xf, height, uncertainty = {'angle':1.0, 'velocity':1.0 , 'height':1.0}):
    acceptable_angles = []
    for angle in range(-91, 91, 1):
        uncert = rangeUncertainty(velocity,uncertainty['velocity'], angle,uncertainty['angle'], height,uncertainty['height'])
        calculatedRange = eval_flight(velocity, angle, height)['range'];
        if abs(calculatedRange - xf) < uncert:
            acceptable_angles.append({'angle': angle, 'uncertainty': uncert})
    return acceptable_angles

In [36]:
uncertainty = {'angle': 1.0, 'velocity': 0.08 , 'height': .025}
hits = hit_the_target(5.5, 1.5, 1.0, uncertainty)

for hit in hits: print(hit)

{'angle': 54, 'uncertainty': 0.3560004306214983}
{'angle': 69, 'uncertainty': 0.08605357092563823}
{'angle': 70, 'uncertainty': 0.08552763306039197}
{'angle': 71, 'uncertainty': 0.08514467274726854}


## P5.3
In preparation for B. Nelson’s “test” of your code, calculate the exit speed
for _all three clicks_ on your launcher. When B. Nelson gives you a target to
hit, you will be free to choose any of the three clicks to hit the target. Now
test your code on several scenarios. Be sure to attempt to hit a target that
is both _above_ and below the launcher. When you feel confident that your
code is working correctly, call B. Nelson over and he will give you a target to
hit. You will then have 1minute to measure the target’s x-y coordinates and
1 minute to determine the correct launch angle and hit the target.

> Below are our targets, our cannon was broken, so we could not fire with `1` click

|Clicks|Height (m)|Exit Velocity (m/s)|
|--|
|1|N/A|N/A|
|2|0.666|3.615|
|3|1.256|4.964|
<sub>Below is the work used to calculate the velocities<sub>


In [37]:
# calculate
click_velocity = [np.sqrt(2*9.81*x) for x in [np.nan, 0.666, 1.256]]
print(click_velocity)

[nan, 3.614819497568309, 4.964143430643397]


### THE TEST

Below are the calculations for Brother Nelson's test. The coordinates are represented in a touple `(x, y)` in meters.

In [38]:
target_coordinates = (1.8, 1.085 - 1.03)
uncertainty = {'angle': 2, 'velocity': 0.08 , 'height': .025}

print ("Calculating clicks and angles needed to hit a target at", target_coordinates)
for click in range(0,3):
    hits = hit_the_target(click_velocity[click], target_coordinates[0], target_coordinates[1], uncertainty)
    print (click+1,"CLICK ---------------- ")
    for hit in hits: print('   ', hit)

Calculating clicks and angles needed to hit a target at (1.8, 0.05499999999999994)
1 CLICK ---------------- 
2 CLICK ---------------- 
3 CLICK ---------------- 
    {'angle': 23, 'uncertainty': 0.16514338007027227}
    {'angle': 24, 'uncertainty': 0.15905759649144183}
    {'angle': 25, 'uncertainty': 0.15335358812656427}
    {'angle': 26, 'uncertainty': 0.1479537270055894}
    {'angle': 27, 'uncertainty': 0.14280361378726933}
    {'angle': 65, 'uncertainty': 0.1272834101529765}
    {'angle': 66, 'uncertainty': 0.1304345169337219}
    {'angle': 67, 'uncertainty': 0.13354042857772386}
    {'angle': 68, 'uncertainty': 0.136588867387097}


> We successfully hit the target `(1.8, 0.055)` meters away with the cannon set at $66.5^\circ$. 

# Take home thoughts

This was the most fun lab so far. I will be honest that I don't like typing math functions, but we were successful so it all pays off. I am glad we took two weeks because we needed it. I am pretty happy with the results.

# Signing off

- Kyle West
- Landon Lewis